**Предисловие:**

**Реализация основана на работе:** ...

**Результаты данной работы представлены в статье (при использовании просьба ссылаться на следующую работу):** ...

**Выборки данных:**
>*    Подписи к изображениям на английском языке из выборки COCO Image Captions
*    Стихи на русском языке с сайта stihi.ru

**Описание работы:**
> В данной работе проведены реализация, обучение и тестирование нейронных сетей для генерации случайных коротких текстов. Используются такие нейронные сети как LSTM и SeqGAN. Проведена оценка качества генерации текста на основе метрики BLEU. Для обучения и тестирования используются следующие выборки данных:
*   Подписи к изображениям на английском языке из выборки COCO Image Captions
*   Стихи на русском языке с сайта stihi.ru




Список всех установленных библиотек и их версии

In [ ]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

Подключение к Google Drive

In [ ]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

path = '/content/drive/My Drive/SeqGAN/'
os.chdir(path)

Объявление параметров по умолчанию

In [ ]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

import helpers # перетащить функции в код

DATASET = 'coco_image_captions' # название выборки данных список выборок: ['stihi_ru', 'coco_image_captions']
CUDA = True # использование cuda (gpu/tpu): True/False
BATCH_SIZE = 500 # количество примеров в партии/батче
MLE_TRAIN_EPOCHS = 20 # количество эпох при обучении генератора на основе MLE
DIS_TRAIN_ITERATIONS = 250 # количество итераций при обучении дискриминатора (итерации содержит несколько эпох, на каждой итерации данные для обучения обновляются)
DIS_TRAIN_EPOCHS = 1 # количество эпох в одной итерации обучения дискриминатора
ADV_TRAIN_EPOCHS = 15 # количество эпох при обучении генератора на основе соревновательного обучения / обучения с подкреплением
POS_NEG_SAMPLES = 10000

GEN_EMBEDDING_DIM = 150 # ширина слоя Embedding генератора
GEN_HIDDEN_DIM = 150 # количество нейронов в слоях генератора
DIS_EMBEDDING_DIM = 150 # ширина слоя Embedding дискриминатора
DIS_HIDDEN_DIM = 150 # количество нейронов в слоях дискриминатора

# параметры START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN и FILE_PATHS задаются автоматически ниже
START_LETTER = None #! стартовое слово (слово которое подается первым на нейронную сеть генератор)
MAX_SEQ_LEN = None # длина генерируемого примера
VOCAB_SIZE = None # количество слов в словаре
FILE_PATHS = None # пути к файлам набора данных
CLOSING_WORD = None # заключительное слово / слово заполнитель ставящееся в конце предложения

# выбор набора данных и инициализация соответствующих глобальных параметров
if DATASET == 'stihi_ru': # выборка с русскими стихами с сайта stihi_ru (длина примера 10 слов, размер словаря 5 000 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 10
  VOCAB_SIZE = 5000
  CLOSING_WORD = 4999#4997
  FILE_PATHS = {'train': r'datasets/stihi_ru/stihi_ru_realtrain_cotra.txt', 'test': r'datasets/stihi_ru/stihi_ru_realtest_coco.txt',
                'vocab': r'datasets/stihi_ru/stihi_ru_vocab_cotra.pkl', 'saved_models': r'saved_models/stihi_ru'}
elif DATASET == 'coco_image_captions': # выборка COCO Image Captions (длина примера 20 слов, размер словаря 4980 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 20
  VOCAB_SIZE = 4838
  CLOSING_WORD = 1814
  FILE_PATHS = {'train': r'datasets/coco_image_captions/coco_image_captions_train.txt', 'test': r'datasets/coco_image_captions/coco_image_captions_test.txt',
                'vocab': r'datasets/coco_image_captions/coco_image_captions_vocab_cotra_test.pkl', 'saved_models': r'saved_models/coco_image_captions'}

Инициализация генераторов случайных чисел.

In [ ]:
# инициализация генераторов случайных чисел.
torch.random.manual_seed(25)
np.random.seed(25)
#системный рандом!!!

Объявление класса Генератора

In [ ]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # количество элементов на скрытом слое
    self.embedding_dim = embedding_dim # размер слоя embedding
    self.max_seq_len = max_seq_len # длина генерируемых примеров
    self.vocab_size = vocab_size # размер словаря использующегося при генерации
    self.gpu = gpu # использование cuda (True/False)
    self.lstm_num_layers = 1 # количество LSTM слоев

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя embeddings
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # объявление LSTM слоев
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # объявление выходного слоя

  def init_hidden(self, batch_size=1):
    # инициализация состояния LSTM слоев
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim)) 
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size

Объявяление класса Дискриминатора

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # размер скрытого слоя
        self.embedding_dim = embedding_dim # ширина слоя embedding
        self.max_seq_len = max_seq_len # длина входного примера
        self.gpu = gpu # использование cuda (True/False)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # объявление LSTM слоев

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # объявление скрытого слоя
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # объявление скрытого слоя

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # объявление выходного слоя

    def init_hidden(self, batch_size):
        # инициализация LSTM слоев
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim
        
        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Объявление функций возвращающих данные

In [ ]:
# функция генерации реалиных данных
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

# функция генерации реалиных данных
def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Функции обучения генератора

In [ ]:
# функция обучения генератора на основе MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # обучение
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # тестирование
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Тестирование генератора на обучающей и тестовой выборках.
  '''
  # тестирование на обучающей
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # тестирование на тестовой
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# функция обучения генератора на основе обучения с подкреплением RL
def train_generator_PG(gen, gen_opt, dis, num_batches): # !переименовать!
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Функция обучения дискриминатора

In [ ]:
# функция обучения дискриминатора
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ДО ОБУЧЕНИЯ: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Функция оценки качества генерации текста

In [ ]:
# оценка качества по BLEU метрике
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# функция оценки качества генерации текста по метрике BLEU
def BLEU(reference_sample, test_sample, print_iteration=100, flag_print=True):
  if flag_print:
    print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  mass_bleu = []
  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0 and flag_print:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))
          
      mass_bleu.append(1.0 * sum(bleu_leakgan) / len(bleu_leakgan))
      if flag_print:
        print('--- --- ---')
        print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")
  return mass_bleu

Функция сохранения моделей

In [ ]:
# функция сохранения моделей (сериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Функция загрузки моделей

In [ ]:
# функция загрузки моделей (десериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
  
  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Загрузка набора данных

In [ ]:
# загрузка словаря
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [ ]:
# загрузка обучающей выборки
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Примеров в обучающей выборке: ", len(data_file_train))

Примеров в обучающей выборке:  80000


In [ ]:
# загрузка тестовой выборки
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Примеров в тестовой выборке: ", len(data_file_test))

Примеров в тестовой выборке:  5000


In [ ]:
# примеры из обучающей выборки
print("Примеры из обучающей выборки")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры из обучающей выборки
# 0 	Пример:  A red stop sign next to the side of a road .                
# 1 	Пример:  A person riding on a bike with a helmet and gloves on .              
# 2 	Пример:  An elephant holding a small piece of paper in its trunk .                
# 3 	Пример:  A train with engine is standing on the railway station .                  
# 4 	Пример:  A person on a snow board in the air at night .                
# 5 	Пример:  A woman on the sidewalk with her back to us .                  
# 6 	Пример:  A gray cat is sitting on the hood of a car .                
# 7 	Пример:  An individual holding a remote control , aiming it at the television and pushing a button .      
# 8 	Пример:  A man holding up a white neck tie with a bucket of paint on it .        
# 9 	Пример:  Two people play tennis on a tennis court in a resort complex .              
# 10 	Пример:  A lighthouse shines brightly in the background of this picture .                  
# 11 	Пример:  A 

Оценка качества примеров из обучающей выборки на основе BLEU  (для сравнения с генератором)

In [ ]:
%%time
# оценка качества на основе примеров из обучающей выборки (для сравнения с генератором)
print("Оценка примеров из обучающей выборки на основе BLEU")
BLEU(data_file_test.tolist(), data_file_train[:500].tolist(), print_iteration=100)

'''
BLEU примеров из обучающей выборки
--- --- ---
Start BLEU
2 100 0.5365504958995001
2 200 0.5426700180034696
2 300 0.5425210147596119
2 400 0.5438925313146432
2 500 0.5499117864293879
--- --- ---
2 -gram BLEU score :  0.5499117864293879
3 100 0.4252853298296578
3 200 0.4294441261529973
3 300 0.42938411299670276
3 400 0.432946142036555
3 500 0.436991427933691
--- --- ---
3 -gram BLEU score :  0.436991427933691
4 100 0.3118228221158724
4 200 0.31347352868277367
4 300 0.3133847969805193
4 400 0.3148005592801212
4 500 0.3173224414085976
--- --- ---
4 -gram BLEU score :  0.3173224414085976
5 100 0.20748007111715946
5 200 0.21057918753505803
5 300 0.20884027925072032
5 400 0.20986772415037908
5 500 0.21083373415979106
--- --- ---
5 -gram BLEU score :  0.21083373415979106
'''

Оценка примеров из обучающей выборки на основе BLEU
--- --- ---
Start BLEU
2 100 0.5421079509296283
2 200 0.5499670537764236
2 300 0.5487035461267846
2 400 0.5475417748156669
2 500 0.5499117864293877
--- --- ---
2 -gram BLEU score :  0.5499117864293877 

3 100 0.4209555782881417
3 200 0.4339168124014302
3 300 0.4329917541110669
3 400 0.4342559012400833
3 500 0.43699142793369145
--- --- ---
3 -gram BLEU score :  0.43699142793369145 

4 100 0.29846429405080027
4 200 0.31175877359006376
4 300 0.3122735814016834
4 400 0.31386833801542907
4 500 0.3173224414085974
--- --- ---
4 -gram BLEU score :  0.3173224414085974 

5 100 0.19259023471054995
5 200 0.20160570942355704
5 300 0.20423897927921025
5 400 0.2063109642063208
5 500 0.210833734159791
--- --- ---
5 -gram BLEU score :  0.210833734159791 

CPU times: user 7min 36s, sys: 127 ms, total: 7min 36s
Wall time: 7min 36s


Создание нейронных сетей генератора и дискриминатора

In [ ]:
# объявление нейронных сетей генератора и дискриминатора, подготовка выборок данных для использования pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) #, lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Обучение генератора на основе MLE

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 8.5058 average_test_NLL = 8.5068
CPU times: user 1.55 s, sys: 46 ms, total: 1.6 s
Wall time: 1.6 s


In [ ]:
%%time
# обучение генератора на основе MLE / предобучение генератора
print('Запуск обучения генератора на основе MLE...')
gen_optimizer = optim.Adam(gen.parameters())#, lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, MLE_TRAIN_EPOCHS) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')
# epoch 1 : .......... average_train_NLL = 1.8447 average_test_NLL = 1.9937

Запуск обучения генератора на основе MLE...
epoch 1 : .......... average_train_NLL = 3.6984 average_test_NLL = 3.0179
epoch 2 : .......... average_train_NLL = 2.6903 average_test_NLL = 2.6763
epoch 3 : .......... average_train_NLL = 2.4632 average_test_NLL = 2.5240
epoch 4 : .......... average_train_NLL = 2.3351 average_test_NLL = 2.4251
epoch 5 : .......... average_train_NLL = 2.2443 average_test_NLL = 2.3538
epoch 6 : .......... average_train_NLL = 2.1714 average_test_NLL = 2.2993
epoch 7 : .......... average_train_NLL = 2.1094 average_test_NLL = 2.2562
epoch 8 : .......... average_train_NLL = 2.0575 average_test_NLL = 2.2122
epoch 9 : .......... average_train_NLL = 2.0134 average_test_NLL = 2.1799
epoch 10 : .......... average_train_NLL = 1.9758 average_test_NLL = 2.1526
epoch 11 : .......... average_train_NLL = 1.9435 average_test_NLL = 2.1228
epoch 12 : .......... average_train_NLL = 1.9156 average_test_NLL = 2.0973
epoch 13 : .......... average_train_NLL = 1.8894 average_test_NLL

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 1.8360 average_test_NLL = 1.9977
CPU times: user 1.55 s, sys: 37 ms, total: 1.58 s
Wall time: 1.59 s


Генерация примеров текстов на основе MLE

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1
# 0 	Пример:  A group of people are parasailing on a laptop in a wooden pool .                                      	Оценка:  [0.5548826333938605, 0.49288159365841444, 0.38126332702591925, 0.26555594605914223]
# 1 	Пример:  A pizza sitting on top of a blue plate with food on it while others look at the table .               	Оценка:  [0.9733285267845752, 0.8334502759861924, 0.6423770871903809, 0.4630352305296693]
# 2 	Пример:  A display of red butter set stands in front of a bed .                                                	Оценка:  [0.533113989983183, 0.42898873304984025, 0.3435599238920265, 0.2806622489471059]
# 3 	Пример:  A couple of people sitting on top of a Wii controller .                                               	Оценка:  [0.4812265031857878, 0.44823751677106155, 0.40342371407615724, 0.3191472754928875]
# 4 	Пример:  A Nintendo Wii on the teams just hit the front window .                                             

Оценка качества генерации текста после обучения с помощью MLE

In [ ]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.5080927823539096
2 200 0.5239255352114979
2 300 0.5196645922747047
2 400 0.5169048486561942
2 500 0.5124157647560529
--- --- ---
2 -gram BLEU score :  0.5124157647560529 

3 100 0.3589668039910406
3 200 0.37453352059029915
3 300 0.37245833636267023
3 400 0.37305866724577763
3 500 0.3688106393597105
--- --- ---
3 -gram BLEU score :  0.3688106393597105 

4 100 0.22814810872756944
4 200 0.24001213438964086
4 300 0.24151700368284398
4 400 0.24360669484432723
4 500 0.24210380591293543
--- --- ---
4 -gram BLEU score :  0.24210380591293543 

5 100 0.1405140568859867
5 200 0.1460657475667092
5 300 0.14816910962004398
5 400 0.15072367072668805
5 500 0.14866956817307805
--- --- ---
5 -gram BLEU score :  0.14866956817307805 

CPU times: user 7min 32s, sys: 109 ms, total: 7min 32s
Wall time: 7min 32s


Оценка с возведением в степень

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1.5
# 0 	Пример:  A man in black shirt playing a video game in a kitchen .                                              	Оценка:  [0.5848976518656018, 0.42362581168373686, 0.30752616970214336, 0.1410852677518539]
# 1 	Пример:  A bunch of people who are sitting in a field .                                                        	Оценка:  [0.5380275868489703, 0.5250407733778476, 0.4754122362294423, 0.39469037777958105]
# 2 	Пример:  A young person flying a kite over a green field .                                                     	Оценка:  [0.45014617508912413, 0.32323518297936166, 0.2111187176080899, 0.10442329545559932]
# 3 	Пример:  A woman in a living room playing a video game .                                                       	Оценка:  [0.4812265031857878, 0.37195938466682343, 0.2789439259471503, 0.1304936956280769]
# 4 	Пример:  A large group of people gathered around a table with a remote controllers .                     

In [ ]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.5554020966946029
2 200 0.5569857332677769
2 300 0.5527045340006904
2 400 0.5512694131771387
2 500 0.5489960772783707
--- --- ---
2 -gram BLEU score :  0.5489960772783707 

3 100 0.48279639356814313
3 200 0.48251694389347527
3 300 0.47611531119061073
3 400 0.47339326248041397
3 500 0.4702095930578137
--- --- ---
3 -gram BLEU score :  0.4702095930578137 

4 100 0.3777182340111975
4 200 0.3805518656873029
4 300 0.3722858666357529
4 400 0.36914010674892966
4 500 0.36686741332932404
--- --- ---
4 -gram BLEU score :  0.36686741332932404 

5 100 0.25688064362212837
5 200 0.26431840820576935
5 300 0.256649993492535
5 400 0.2554217947897245
5 500 0.25457816646887266
--- --- ---
5 -gram BLEU score :  0.25457816646887266 

CPU times: user 7min 36s, sys: 117 ms, total: 7min 36s
Wall time: 7min 36s


Предварительное обучение дискриминатора

In [ ]:
%%time
# предобучение дискриминатора
print('Запуск обучения дискриминатора...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # регуляризация
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)#25, 1 | (15, 3), (25, 1)

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ДО ОБУЧЕНИЯ: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Запуск обучения дискриминатора...
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 1 epoch 1 : .......... average_loss = 0.3886, train_acc = 0.8716, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 2 epoch 1 : .......... average_loss = 0.3226, train_acc = 0.8887, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 3 epoch 1 : .......... average_loss = 0.3019, train_acc = 0.8909, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 4 epoch 1 : .......... average_loss = 0.2958, train_acc = 0.8903, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 5 epoch 1 : .......... average_loss = 0.2889, train_acc = 0.8902, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 6 epoch 1 : .......... average_loss = 0.2957, train_acc = 0.8880, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 7 epoch 1 : .......... average_loss = 0.2897, train_acc = 0.8888, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 8 epoch 1 : .......... average_loss = 0.2869, train_acc = 0.8885, val_acc = 0.5140
ДО ОБУ

Состязательное обучение. Обучение генератора на основе обучения с подкреплением

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 1.8360 average_test_NLL = 1.9977


In [ ]:
%%time
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# состязательное обучение генератора
print('\nStarting Adversarial Training...')

for epoch in range(ADV_TRAIN_EPOCHS):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # обучение генератора
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)#

    # тестирование nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # обучение дискриминатора
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

RuntimeError: ignored

Тестирование генератора на обучающей выборке после состязательного обучения.

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.5186 average_test_NLL = 2.0730
# average_train_NLL = 1.4459 average_test_NLL = 1.9926

Генерация примеров текстов на основе SeqGAN

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Оценка качества генерации на основе SeqGAN

In [ ]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка с возведением в степень

In [ ]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN с возведением в степень")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

In [ ]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Загрузка сохраненной модели

In [ ]:
# загрузка моделей
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()

Оценка разнообразия текста

In [ ]:
degree = 1.5

In [ ]:
# плагиат из обучающей выборки
samples = gen.sample(1500, degree=degree).cpu().detach().numpy().tolist()
# samples = data_file_test.tolist()[:500]
train_samples = data_file_train.tolist()
n = 0
for i in range(len(samples)):
  if samples[i] in train_samples:
    n += 1
  if i%(len(samples)//10) == 0:
    print(i/len(samples)*100, "%")
print("Совпало сгенерированных примеров с обучающей выборкой: ", n, "из", len(samples))
print("Плагиат: ", n/len(samples)*100, "%")

In [ ]:
# оригинальность примеров
N_samples = 1500
samples_1 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()
samples_2 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

n = 0
for i in range(len(samples_1)):
  if samples_1[i] in samples_2:
    n += 1
  if i%(len(samples_1)//10) == 0:
    print(i/len(samples_1)*100, "%")

print("Оригинальность генерируемых примеров")
print("Совпало примеров из выборки 1 с примерами из выборки 2: ", n, "из", len(samples_1))
print("Плагиат: ", n/len(samples_1)*100, "%")
print("Оригинальность: ", (1-n/len(samples_1))*100, "%")

In [ ]:
# оригинальность примеров подход 2
N_samples = 1500
samples = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

i = 0
n = 0
while len(samples)>0:
  if samples[0] not in samples[1:]:
    n += 1
  if i%(N_samples//10) == 0:
    print(i/N_samples*100, "%")
  del(samples[0])
  i += 1

print("Оригинальность генерируемых примеров")
print("Сгенерировано оригинальных примеров: ", n, "из", N_samples)
print("Оригинальность: ", n/N_samples*100, "%")
print("Плагиат: ", (1-n/N_samples)*100, "%")